In [48]:
import pandas as pd
import csv

In [49]:
def compare_answers(answer_train, answer_test):
    if answer_train == "Sim" and answer_test == "Sim":
        return 1, 0, 0, 0  # TP, TN, FN, FP
    elif answer_train == "Não" and answer_test == "Não":
        return 0, 1, 0, 0  # TP, TN, FN, FP
    elif answer_train == "Sim" and answer_test == "Não":
        return 0, 0, 1, 0  # TP, TN, FN, FP
    elif answer_train == "Não" and answer_test == "Sim":
        return 0, 0, 0, 1  # TP, TN, FN, FP
    elif(answer_test == answer_train):
        return 1, 0, 0, 0  # TP, TN, FN, FP
    elif(answer_test != answer_train):
        return 0, 0, 0, 1  # TP, TN, FN, FP
    return 0, 0, 0, 0

def calculate_metrics(df, columns):
    
    metrics = {col: {'TP': 0, 'TN': 0, 'FN': 0, 'FP': 0} for col in columns}

    for i in range(len(df)):
        for col in columns:
            answer_train = df[col + "_x"].iloc[i]
            answer_test = df[col + "_y"].iloc[i]

            TP, TN, FN, FP = compare_answers(answer_train, answer_test)
            metrics[col]['TP'] += TP
            metrics[col]['TN'] += TN
            metrics[col]['FN'] += FN
            metrics[col]['FP'] += FP

    return metrics
    
def calculate_precision_recall_f1(metrics):
    results = {}
    for col, counts in metrics.items():
        TP = counts['TP']
        TN = counts['TN']
        FN = counts['FN']
        FP = counts['FP']

        precision = TP / (TP + FP) if TP + FP != 0 else 0
        recall = TP / (TP + FN) if TP + FN != 0 else 0
        f1_score = 2 * ((precision * recall) / (precision + recall)) if precision + recall != 0 else 0

        results[col] = {'precision': precision, 'recall': recall, 'f1_score': f1_score}
    
    return results

def save_metrics(csv_filename, results):
    with open(csv_filename, encoding='utf-8', mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Coluna", "Precision", "Recall", "F1 Score"])
        for col, result in results.items():
            writer.writerow([col, result['precision'], result['recall'], result['f1_score']])



In [50]:
"""
columns = [
    ("O nódulo é sólido?", "O nódulo é sólido?"),
    ("O nódulo tem densidade semissólida, parcialmente sólida , ou em partes moles?", "O nódulo é em partes moles, semissólido ou subsólido?"),
    ("O nódulo tem densidade em vidro fosco?", "O nódulo é em vidro fosco?"),
    ("O nódulo tem borda espiculada e/ou mal definida?", "O nódulo é espiculado, irregular ou mal definido?"),
    ("O nódulo é calcificado?", "O nódulo é calcificado?"),
    ("Localização do nódulo", "Localização do nódulo"),
    ("Tamanho do nódulo", "Tamanho do nódulo")
]

#df_train = pd.read_csv(r"data\all_lung_nodule\few_shot\test_mod.csv")
#df_test = pd.read_csv(r"data\all_lung_nodule\zero_shot\results_gemini\results_prompt_1_structured.csv")
#csv_filename = r"data\all_lung_nodule\zero_shot\results_gemini\metrics_prompt_1_structured.csv"
"""
columns = [
    "O nódulo é sólido?",
    "O nódulo é em partes moles, semissólido ou subsólido?",
    "O nódulo é em vidro fosco?",
    "O nódulo é espiculado, irregular ou mal definido?",
    "O nódulo é calcificado?",
    "Localização do nódulo",
    "Tamanho do nódulo"
]
df_train = pd.read_csv(r"C:\Users\tarcisio.ferreira\Desktop\llms_lung\data\all_lung_nodule\few_shot\test_mod.csv")
df_test = pd.read_csv(r"C:\Users\tarcisio.ferreira\Desktop\llms_lung\data\all_lung_nodule\zero_shot\results_llama3\results_prompt_2_structured.csv", index_col=0)
csv_filename = r"C:\Users\tarcisio.ferreira\Desktop\llms_lung\data\all_lung_nodule\zero_shot\results_llama3\metrics_prompt_2_structured.csv"

In [51]:
df_train = df_train.rename(columns={'Laudo': 'ID do laudo'})

df_train

,ID do laudo,O nódulo é sólido?,"O nódulo é em partes moles, semissólido ou subsólido?",O nódulo é em vidro fosco?,"O nódulo é espiculado, irregular ou mal definido?",O nódulo é calcificado?,Localização do nódulo,Tamanho do nódulo
0,863.0,Não,Não,Não,Não,Sim,LSD,Não
1,864.0,Não,Não,Não,Não,Não,LIE,"0,5 cm"
2,865.0,Não,Não,Não,Não,Sim,LIE,"0,28 x 0,23 cm"
3,866.0,Não,Não,Não,Não,Não,Não,Não
4,867.0,Sim,Não,Não,Não,Não,LM,"1,0 x 0,9 cm"
...,...,...,...,...,...,...,...,...
110,958.0,Não,Não,Não,Não,Não,Não,Não
111,959.0,Não,Não,Não,Não,Sim,LSE,Não
112,960.0,Não,Não,Não,Não,Sim,LID,"0,2 cm"
113,961.0,Não,Não,Não,Não,Sim,LID,"0,4 cm"


In [52]:
df_test

,Nódulo,ID do laudo,O nódulo é sólido?,"O nódulo é em partes moles, semissólido ou subsólido?",O nódulo é em vidro fosco?,"O nódulo é espiculado, irregular ou mal definido?",O nódulo é calcificado?,Localização do nódulo,Tamanho do nódulo
0,0,863.0,Não,Não,Não,Não,Sim,LSD,Não
0,1,864.0,Não,Não,Não,Não,Não,LIE,"0,5 cm"
0,2,865.0,Não,Não,Não,Não,Sim,LIE,"0,28 x 0,23 cm"
0,3,866.0,Não,Não,Não,Não,Não,LSE,"0,3 cm"
0,4,867.0,Sim,Não,Não,Não,Não,LM,"1,0 x 0,9 cm"
...,...,...,...,...,...,...,...,...,...
0,121,958.0,Não,Não,Não,Não,Não,Não,Não
0,122,959.0,Não,Não,Não,Não,Sim,LSE,Não
0,123,960.0,Não,Não,Não,Não,Sim,LID,"0,2 cm"
0,124,961.0,Não,Não,Não,Não,Sim,LID,"0,4 cm"


In [ ]:
# Encontrando as linhas que estão em df1 mas não em df2
unique_df_train = df_train[~df_train['ID do laudo'].isin(df_test['ID do laudo'])]
# Encontrando as linhas que estão em df2 mas não em df1
unique_df_test = df_test[~df_test['ID do laudo'].isin(df_train['ID do laudo'])]

# Contando as linhas exclusivas
count_unique_df1 = unique_df_train.shape[0]
count_unique_df2 = unique_df_test.shape[0]

print(f"Linhas exclusivas em df1: {count_unique_df1}")
print(f"Linhas exclusivas em df2: {count_unique_df2}")

print(unique_df_train)
print(unique_df_test)

In [54]:
intersection_df = pd.merge(df_train, df_test, how='inner', on=['ID do laudo'])


In [55]:
intersection_df

,ID do laudo,O nódulo é sólido?_x,"O nódulo é em partes moles, semissólido ou subsólido?_x",O nódulo é em vidro fosco?_x,"O nódulo é espiculado, irregular ou mal definido?_x",O nódulo é calcificado?_x,Localização do nódulo_x,Tamanho do nódulo_x,Nódulo,O nódulo é sólido?_y,"O nódulo é em partes moles, semissólido ou subsólido?_y",O nódulo é em vidro fosco?_y,"O nódulo é espiculado, irregular ou mal definido?_y",O nódulo é calcificado?_y,Localização do nódulo_y,Tamanho do nódulo_y
0,863.0,Não,Não,Não,Não,Sim,LSD,Não,0,Não,Não,Não,Não,Sim,LSD,Não
1,864.0,Não,Não,Não,Não,Não,LIE,"0,5 cm",1,Não,Não,Não,Não,Não,LIE,"0,5 cm"
2,865.0,Não,Não,Não,Não,Sim,LIE,"0,28 x 0,23 cm",2,Não,Não,Não,Não,Sim,LIE,"0,28 x 0,23 cm"
3,866.0,Não,Não,Não,Não,Não,Não,Não,3,Não,Não,Não,Não,Não,LSE,"0,3 cm"
4,867.0,Sim,Não,Não,Não,Não,LM,"1,0 x 0,9 cm",4,Sim,Não,Não,Não,Não,LM,"1,0 x 0,9 cm"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,958.0,Não,Não,Não,Não,Não,Não,Não,121,Não,Não,Não,Não,Não,Não,Não
111,959.0,Não,Não,Não,Não,Sim,LSE,Não,122,Não,Não,Não,Não,Sim,LSE,Não
112,960.0,Não,Não,Não,Não,Sim,LID,"0,2 cm",123,Não,Não,Não,Não,Sim,LID,"0,2 cm"
113,961.0,Não,Não,Não,Não,Sim,LID,"0,4 cm",124,Não,Não,Não,Não,Sim,LID,"0,4 cm"


In [ ]:
metrics = calculate_metrics(intersection_df, columns)
results = calculate_precision_recall_f1(metrics)
save_metrics(csv_filename, results)

for col, result in results.items():
    print(f"{col}: Precision: {result['precision']}, Recall: {result['recall']}, F1 Score: {result['f1_score']}")